In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from google.colab import files
from google.colab import drive
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/T_no_missing.csv'

# Load the CSV data into a DataFrame
data = pd.read_csv(file_path)
data.head(10)

#remove country for later calculation
data = data.drop(columns=["Country"])
data

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.820,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069
181,0.184,0.348,0.507,0.835,0.355,0.656,0.993,0.783,0.728,0.834,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891


In [3]:
miss_data = pd.read_csv('/content/drive/MyDrive/T_missing.csv')
miss_data_x = miss_data.drop(columns=["Country"])
miss_data_x

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,NaN,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069
181,0.184,0.348,0.507,0.835,NaN,0.656,0.993,0.783,0.728,NaN,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891


# Mean imputation - row-wise

In [4]:
# Transpose the data so rows become columns
miss_data_x_transposed = miss_data_x.T
miss_data_x_transposed

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
SimpleImputer()
imputed_mean_data_x_transposed = imp.fit_transform(miss_data_x_transposed)
imputed_mean_data_x = imputed_mean_data_x_transposed.T

#put the titles back, convert it into dataframe
imputed_mean_data_x_df = pd.DataFrame(imputed_mean_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

imputed_mean_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.993000,1.311000,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498000
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.065000,1.532000,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498000
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,1.217357,1.856000,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926000
3,0.386,0.174,1.508,1.279,0.570000,1.788,1.018,1.055,1.050000,1.480000,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562000
4,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.169000,0.295000,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,0.759929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.249000,0.855000,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455000
180,0.529,0.315,0.903,1.356,1.353000,1.536,1.415,0.935,0.874000,1.296000,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069000
181,0.184,0.348,0.507,0.835,1.009458,0.656,0.993,0.783,0.728000,1.009458,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711000
182,0.544,-0.117,0.362,0.761,0.393000,0.339,0.876,0.064,-0.040000,0.263000,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891000


# Calculate the MSE, RMSE, MAE of the Mean Imputation Method (Row)

In [5]:
data=data.to_numpy()
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_mean_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of mean imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of mean imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of mean imputation: {mae}")

Mean Squred Error (MSE) of mean imputation: 0.2771681668089272
Root Mean Squred Error (RMSE) of mean imputation: 0.5264676313021791
Mean Absolute Error (MAE) of mean imputation: 0.3821997763586927


In [7]:
print(imputed_values)

[0.93778571 1.21735714 0.75992857 0.68592308 0.68592308 0.68592308
 0.52611111 0.52611111 0.96659259 0.96659259 0.77414286 1.05418519
 1.05418519 0.70148148 0.70148148 1.32614815 1.32614815 0.43007407
 0.43007407 1.52961538 1.52961538 1.52961538 0.81385714 0.79507407
 0.79507407 0.73796429 1.17862963 1.17862963 0.41665385 0.41665385
 0.41665385 0.93896296 0.93896296 0.70374074 0.70374074 0.82507407
 0.82507407 0.76707407 0.76707407 0.74428571 1.155      1.155
 0.75788889 0.75788889 0.37825926 0.37825926 0.72321429 0.99453846
 0.99453846 0.99453846 0.71030769 0.71030769 0.71030769 0.46871429
 0.82189286 0.97780769 0.97780769 0.97780769 0.86796    0.86796
 0.86796    0.86796    1.25677778 1.25677778 0.6825     0.96310714
 0.82614815 0.82614815 0.77210714 0.67285185 0.67285185 1.56396296
 1.56396296 0.73489286 0.74359259 0.74359259 1.42528571 1.2975
 0.61271429 0.73974074 0.73974074 1.18367857 0.91889286 0.8752963
 0.8752963  1.09751852 1.09751852 0.7332963  0.7332963  1.08533333
 1.08533

# Median imputation - Row-wise

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
SimpleImputer()
imputed_median_data_x_transposed = imp.fit_transform(miss_data_x_transposed)
imputed_median_data_x = imputed_median_data_x_transposed.T

#put the titles back, convert it into dataframe
imputed_median_data_x_df = pd.DataFrame(imputed_median_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

imputed_median_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,1.201,1.856,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.050,1.480,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,0.744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.874,1.296,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069
181,0.184,0.348,0.507,0.835,1.007,0.656,0.993,0.783,0.728,1.007,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891


# Calculate the MSE, RMSE, MAE of the Median Imputation Method (Row)

In [ ]:
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_median_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of median imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of median imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of median imputation: {mae}")

Mean Squred Error (MSE) of median imputation: 0.28004135456273765
Root Mean Squred Error (RMSE) of median imputation: 0.5291893371589583
Mean Absolute Error (MAE) of median imputation: 0.3829334600760456


# Multivariate feature imputation - row-wise

In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
IterativeImputer(random_state=0)
multivariate_impute_data_x_transposed = imp.fit_transform(miss_data_x_transposed)
multivariate_impute_data_x = multivariate_impute_data_x_transposed.T

#put the titles back, convert it into dataframe
multivariate_impute_data_x_df = pd.DataFrame(multivariate_impute_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

#print out the dataframe after imputing the
multivariate_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.993000,1.311000,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.498000
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.065000,1.532000,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.498000
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,0.857199,1.856000,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.926000
3,0.386,0.174,1.508,1.279,0.570000,1.788,1.018,1.055,1.050000,1.480000,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.562000
4,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.169000,0.295000,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.328749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.249000,0.855000,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.455000
180,0.529,0.315,0.903,1.356,1.353000,1.536,1.415,0.935,0.874000,1.296000,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.069000
181,0.184,0.348,0.507,0.835,0.406726,0.656,0.993,0.783,0.728000,0.856499,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.711000
182,0.544,-0.117,0.362,0.761,0.393000,0.339,0.876,0.064,-0.040000,0.263000,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.891000


# Calculate the MSE, RMSE, MAE of the multivariate feature Imputation Method (row)

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = multivariate_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of multivariate feature imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of multivariate feature imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of multivariate feature imputation: {mae}")

Mean Squred Error (MSE) of multivariate feature imputation: 0.06108600421672435
Root Mean Squred Error (RMSE) of multivariate feature imputation: 0.2471558298254855
Mean Absolute Error (MAE) of multivariate feature imputation: 0.18585180792928024


# Nearest neighbors imputation - row-wise

In [ ]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")
KNN_impute_data_x_transposed = imputer.fit_transform(miss_data_x_transposed)
KNN_impute_data_x = KNN_impute_data_x_transposed.T

#put the titles back, convert it into dataframe
KNN_impute_data_x_df = pd.DataFrame(KNN_impute_data_x, columns=['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999',
       'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007',
       'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015',
       'T2016', 'T2017', 'T2018', 'T2019', 'T2020'])

#print out the dataframe after imputing the
KNN_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,T2011,T2012,T2013,T2014,T2015,T2016,T2017,T2018,T2019,T2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.9930,1.3110,...,1.397,0.223,1.281,0.456,1.093,1.555,1.540,1.544,0.910,0.4980
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.0650,1.5320,...,1.055,1.487,1.333,1.198,1.569,1.464,1.121,2.028,1.675,1.4980
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,1.5205,1.8560,...,1.398,1.147,1.192,1.690,1.121,1.757,1.512,1.210,1.115,1.9260
3,0.386,0.174,1.508,1.279,0.570,1.788,1.018,1.055,1.0500,1.4800,...,1.677,1.265,0.831,1.946,1.690,1.990,1.925,1.919,1.964,2.5620
4,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.1690,0.2950,...,0.880,0.552,1.044,0.828,1.331,1.609,0.870,1.395,1.752,1.6805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.2490,0.8550,...,0.581,1.043,1.114,0.863,1.326,1.615,0.735,2.007,1.204,1.4550
180,0.529,0.315,0.903,1.356,1.353,1.536,1.415,0.935,0.8740,1.2960,...,1.579,1.344,1.423,1.401,1.510,1.732,2.204,0.942,1.477,2.0690
181,0.184,0.348,0.507,0.835,0.502,0.656,0.993,0.783,0.7280,0.8745,...,0.921,1.074,1.016,1.053,1.412,1.660,1.429,1.290,1.444,1.7110
182,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.0400,0.2630,...,0.956,0.972,0.790,0.917,1.450,1.401,0.105,0.648,0.855,0.8910


# Calculate the MSE, RMSE, MAE of the KNN Imputation Method (row)

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = KNN_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of KNN imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of KNN imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of KNN imputation: {mae}")

Mean Squred Error (MSE) of KNN imputation: 0.16774568060836503
Root Mean Squred Error (RMSE) of KNN imputation: 0.4095676752483831
Mean Absolute Error (MAE) of KNN imputation: 0.2998403041825095
